In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Assignment II-a

**Group 16**
George Cotea, 13842013; Laura Vochita, 13982540; Bartek Hryniecki, 15286002; Alexander Hepburn, 13175017

In [2]:
data = pd.read_csv("./Data/data.csv",header=[1])

FileNotFoundError: [Errno 2] No such file or directory: './Data/data.csv'

## Data Preprocessing

In [ ]:
print(data.dtypes)

Seeing that some that the categorical variables were not imported as such, we have to encode them properly.

In [ ]:
#Creating dummy variables
data.SEX = data.SEX.astype("category")
data.EDUCATION = data.EDUCATION.astype('category')
data.MARRIAGE = data.MARRIAGE.astype('category')
data.PAY_0 = data.PAY_0.astype('category')
data.PAY_2 = data.PAY_2.astype('category')
data.PAY_3 = data.PAY_3.astype('category')
data.PAY_4 = data.PAY_4.astype('category')
data.PAY_5 = data.PAY_5.astype('category')
data.PAY_6 = data.PAY_6.astype('category')

pd.get_dummies(data)

In [ ]:
print(data.dtypes)

In [ ]:
print(data.info())

In [ ]:
print(data.describe())

## Experimenting with scalers

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score,KFold, learning_curve, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC

In [ ]:
#Splitting the data
X = data.drop(['ID','default payment next month'],axis=1)
y = data['default payment next month']
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=99, shuffle=True,stratify=y)
print(X_train.shape,y_train.shape)

In [ ]:
X_train.hist(figsize=(15,15), bins=100)
plt.show()

### kNN

In [ ]:
kf = KFold(n_splits=10, shuffle=True, random_state=99)
#K-nearest neighbour
#Standard Scaler
steps = [('scaler',StandardScaler()),('model',KNeighborsClassifier())]
pipeline_knn_sdscaler = Pipeline(steps)
pipeline_knn_sdscaler.fit(X_train,y_train)
print('Train set score:',pipeline_knn_sdscaler.score(X_train,y_train))
print('Test set score:',pipeline_knn_sdscaler.score(X_test,y_test))



In [ ]:
#K-nearest neighbour
#MinMax Scaler
steps = [('scaler',MinMaxScaler()),('model',KNeighborsClassifier(n_neighbors=10))]
pipeline_knn_minmax = Pipeline(steps)
pipeline_knn_minmax.fit(X_train,y_train)
print('Train set score:',pipeline_knn_minmax.score(X_train,y_train))
print('Test set score:',pipeline_knn_minmax.score(X_test,y_test))

We note that in this case of KNN, the MinMax scaler outperforms the Standard Scaler.

### Logistic regression

In [ ]:
###Logistic Regression
#Standard Scaler
steps = [('scaler',StandardScaler()),('model',LogisticRegression(max_iter=5000))]
pipeline_logreg_sdscaler = Pipeline(steps)
pipeline_logreg_sdscaler.fit(X_train,y_train)
print('Train set score:',pipeline_logreg_sdscaler.score(X_train,y_train))
print('Test set score:',pipeline_logreg_sdscaler.score(X_test,y_test))


In [ ]:
###Logistic Regression
#MinMax Scaler
steps = [('scaler',MinMaxScaler()),('model',LogisticRegression(max_iter=5000))]
pipeline_logreg_minmax = Pipeline(steps)
pipeline_logreg_minmax.fit(X_train,y_train)
print('Train set score:',pipeline_logreg_minmax.score(X_train,y_train))
print('Test set score:',pipeline_logreg_minmax.score(X_test,y_test))

We note that in this case of logistic regression, the min-max scaler outperforms the standard scaler.

### Linear SVC

In [ ]:
#Linear SVC
#Standard Scaler
steps = [('scaler',StandardScaler()),('model',LinearSVC(dual=False))]
pipeline_svc_sdscaler = Pipeline(steps)
pipeline_svc_sdscaler.fit(X_train,y_train)

print('Train set score:',pipeline_svc_sdscaler.score(X_train,y_train))
print('Test set score:',pipeline_svc_sdscaler.score(X_test,y_test))

In [ ]:
#Linear SVC
#MinMax Scaler
steps = [('scaler',MinMaxScaler()),('model',LinearSVC(dual=False))]
pipeline_svc_minmax = Pipeline(steps)
pipeline_svc_minmax.fit(X_train,y_train)

print('Train set score:',pipeline_svc_minmax.score(X_train,y_train))
print('Test set score:',pipeline_svc_minmax.score(X_test,y_test))

#### Concluding the scaling experiment
We have established that the minmax scaler performs somewhat better (Test score: 0.8084) than the standard scaler (Test score: 0.7919). The same cannot be said about the logistic regression and linear svr models where scaling the data with either tool does not result in significantly different scores.

 
We will now proceed to establish the optimal hyperparameters for our models.

## Modelling

In [ ]:
kf = KFold(n_splits=10, shuffle=True, random_state=99)

In [ ]:
#K-nearest neighbour
steps = [('scaler',MinMaxScaler()),('model',KNeighborsClassifier())]
pipeline_knn_st_scaler = Pipeline(steps)

param_grid = {'model__n_neighbors':range(1,20)}
grid = GridSearchCV(pipeline_knn_st_scaler, param_grid=param_grid, cv=kf)
grid.fit(X_train,y_train)

In [ ]:
print(grid.best_params_)

In [ ]:
#Logistic regression
steps = [('scaler',MinMaxScaler()),('model',LogisticRegression())]
pipeline_logreg = Pipeline(steps)

param_grid = {'model__C':np.logspace(-3,4,150),'model__max_iter':[1000,5000,10000,15000]}
grid = GridSearchCV(pipeline_logreg,param_grid=param_grid,cv=kf)
grid.fit(X_train,y_train)

In [ ]:
print(grid.best_params_)

In [ ]:
#Linear SVC
steps = [('scaler',MinMaxScaler()),('model',LinearSVC(dual=False))]
pipeline_svc = Pipeline(steps)

param_grid = {'model__C':np.logspace(-3,4,150)}
grid = GridSearchCV(pipeline_svc,param_grid=param_grid,cv=kf)
grid.fit(X_train,y_train)

In [ ]:
print(grid.best_params_)

Having established the optimal hyperparameters we now use them to train the three models.

In [ ]:
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error

In [ ]:
#K-nearest neighbour
kf = KFold(n_splits=10, shuffle=True, random_state=99)

steps = [('scaler',MinMaxScaler()),('model',KNeighborsClassifier(n_neighbors=13))]
pipeline_knn = Pipeline(steps)
pipeline_knn.fit(X_train,y_train)

In [ ]:
print("Training set score: {:.3f}".format(pipeline_knn.score(X_train, y_train)))
print("Test set score: {:.3f}".format(pipeline_knn.score(X_test, y_test)))

In [ ]:
#Logistic Regression
steps = [('scaler',MinMaxScaler()),('model',LogisticRegression(C=4208.826990208898,max_iter=1000))]
pipeline_logreg = Pipeline(steps)
pipeline_logreg.fit(X_train,y_train)

In [ ]:
print("Training set score: {:.3f}".format(pipeline_logreg.score(X_train, y_train)))
print("Test set score: {:.3f}".format(pipeline_logreg.score(X_test, y_test)))

In [ ]:
#Linear SVC
steps = [('scaler',MinMaxScaler()),('model',LinearSVC(C=3.7193735530726437,dual=False))]
pipeline_svc = Pipeline(steps)
pipeline_svc.fit(X_train, y_train)

In [ ]:
print("Training set score: {:.3f}".format(pipeline_svc.score(X_train, y_train)))
print("Test set score: {:.3f}".format(pipeline_svc.score(X_test, y_test)))

## Evaluation



In [ ]:
# Define a function that evaluates the model and returns a full spectrum of required evaluation metrics
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def evaluate (est,X_test,y_test):
    
    y_pred = est.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    
    #Macro-precision, f1-score
    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(y_test, y_pred, average='macro')

    #Micro-precision, f1-score
    precision_micro, recall_micro, f1_micro, _ = precision_recall_fscore_support(y_test, y_pred, average='micro')
    
    print(' *** MODEL EVALUATION REPORT ***\n\n','Accuracy: {:.3f}\n'.format(accuracy),'Macro-Precision: {:.3f}\n'.format(precision_macro),'Macro-Recall: {:.3f}\n'.format(recall_macro),'Macro-F1: {:.3f}\n\n'.format(f1_macro),'Micro-Precision: {:.3f}\n'.format(precision_micro),'Micro-Recall: {:.3f}\n'.format(recall_micro),'Micro-F1: {:.3f}\n'.format(f1_micro))



In [ ]:
#kNN - Evaluation
evaluate(pipeline_knn,X_test,y_test)

In [ ]:
#Logistic regression - Evaluation
evaluate(pipeline_logreg,X_test,y_test)

In [ ]:
#SVC - Evaluation
evaluate(pipeline_svc,X_test,y_test)

### Interpreting the evaluation metrics

We note that in terms of **accuracy** the three models perform quite similarly, with Logistic Regression being slightly better with a score of 0.810. **Accuracy** is a metric which computes the ratio of correctly predicted instances to the total number of instances, therefore we can say that the Logistic Regression model correctly predicted more instances.

In terms of the macro metrics we observe a similar trend, however this time the SVC model has a slightly higher **macro-precision** score of 0.770. This suggests that the SVC model is performs better true positives predictions by the total number of positive predictions on average across all classes.
The highest **macro-recall** score can be found in the KNN model at 0.635, which entails that this model is slightly better at predicting the true positives by the actual number of positives in the dataset, on average across all classes.
Lastly, the **F1** score is the highest for the KNN model as well, of 0.656. From this, we can conclude that the KNN algorithm provides the best balance between precision and recall, and might be the most suitable to deal with unevenly distributed data across all classes in the dataset.

The micro metrics consider the overall performance of the employed models, without distinguishing between individual classes.
The Logistic Regression model has a slightly higher **micro-precision** score, of 0.810, which means that when treating the problem as a binary classification problem, this model has a slightly higher ratio of true positives to the total number of positive predictions, across all classes.
The **micro-recall** score is the highest for Logistic Regression as well (0.810), which further supports the choice of this model for a binary classification problem on this dataset, as we can observe that it also showcases a better ratio between the true positives and the actual number of positives in the dataset, across all classes.
Lastly, Logistic Regression also presents the highest **micro-F1** score of 0.810, which means that this model also has the best balance between precision and recall, across all datasets.

Finally, we can also note that the models, especially Logistic regression, tend to perform better based on the micro metrics, which is to be expected since we have a binary target variable.

# Assignment II-B


In [ ]:
#Splitting the data
X = data.drop(['ID','default payment next month'],axis=1)
y = data['default payment next month']
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=99, shuffle=True,stratify=y)
print(X_train.shape,y_train.shape)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

kf = KFold(n_splits=10, shuffle=True, random_state=99)

### Decision tree

In [ ]:
dtree = DecisionTreeClassifier()
dtree.fit(X_train,y_train)


In [ ]:
print("Training set score: {:.3f}".format(dtree.score(X_train, y_train)))
print("Test set score: {:.3f}".format(dtree.score(X_test, y_test)))

Without carefully selected hyperparameters the tree classifier will always end up overfitting. We can observe this from the above result, seeing that the training score nears 100%, but the test score lags behind at 72.5%.

In [ ]:
dtree = DecisionTreeClassifier()
param_grid = {'max_depth':range(1,7),'max_features':range(1,10)}
grid = GridSearchCV(dtree,param_grid=param_grid,cv=kf)
grid.fit(X_train,y_train)

In [ ]:
print(grid.best_params_)

In [ ]:
dtree = DecisionTreeClassifier(max_depth=6,max_features=7)
dtree.fit(X_train,y_train)

In [ ]:
print("Training set score: {:.3f}".format(dtree.score(X_train, y_train)))
print("Test set score: {:.3f}".format(dtree.score(X_test, y_test)))

#### Plot

In [ ]:
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt

# Plotting the best decision tree
plt.figure(figsize=(20, 10))
plot_tree(dtree,
          filled=True,
          rounded=True,
          class_names=['Not Default', 'Default'],
          feature_names=X_train.columns,
          max_depth=3)
plt.title("Decision Tree Visualization")
plt.savefig('decision_tree.pdf')
plt.show()

In [ ]:
# Plot the feature importances
plt.figure(figsize=(20, 10))
plt.bar(X_train.columns, dtree.feature_importances_)
plt.xticks(rotation=45, ha='right', fontsize=14)
plt.title("Feature Importances")
plt.show()

### Random Forest

In [ ]:
forest = RandomForestClassifier()
forest.fit(X_train,y_train)

In [ ]:
print("Training set score: {:.3f}".format(forest.score(X_train, y_train)))
print("Test set score: {:.3f}".format(forest.score(X_test, y_test)))

In [ ]:
forest = RandomForestClassifier()
param_grid = {
    'n_estimators': [100, 150, 200],  
    'max_depth': [3, 4, 5],  
    'max_features': [0.5, 0.7, 'sqrt', 'log2']  
}
grid = GridSearchCV(forest,param_grid=param_grid,cv=3, scoring='accuracy',n_jobs=-1,verbose=1)
grid.fit(X_train,y_train)

In [ ]:
print(grid.best_params_)

In [ ]:
forest = RandomForestClassifier(n_estimators=200,max_features=0.5,max_depth=5)
forest.fit(X_train,y_train)

In [ ]:
print("Training set score: {:.3f}".format(forest.score(X_train, y_train)))
print("Test set score: {:.3f}".format(forest.score(X_test, y_test)))

#### Plot

In [ ]:
plt.figure(figsize=(20, 10))
plot_tree(forest.estimators_[0], filled=True, feature_names=X_train.columns, class_names=['0', '1'], rounded=True)
plt.title('Final Decision Tree in Random Forest')
plt.savefig('random_forest.pdf')
plt.show()

In [ ]:
# Plot the feature importances
plt.figure(figsize=(20, 10))
plt.bar(X_train.columns, forest.feature_importances_)
plt.xticks(rotation=45, ha='right', fontsize=14)
plt.title("Feature Importances")
plt.show()

## Evaluation II-B

In [ ]:
evaluate(dtree,X_test,y_test)

In [ ]:
evaluate(forest,X_test,y_test)

### Interpreting the evaluation metrics

We note that the **accuracy** is slightly higher for the decision tree with a score of 0.819. This suggests that the decision tree correctly predicted 81.9% of the instances.

On a macro level we look at the average scores across all classes, while accounting for individual classes.
The **precision** score here is slightly the higher for Random Forest (0.758), which means that this model has a better ratio of true positives to all predicted positives.
Moreover, the Random Forest model also has a higher **recall** score of 0.659. This suggests that this model has a better ratio of true positives predicted to the actual number of positives in the dataset.
Lastly, the **F1** score is also higher for Random Forest (0.681), which means that this model has a better balance between precision and recall, across all classes.

On a micro level, we look at the overall performance of the two models, without considering each individual class.
In this case, the highest **precision**, **recall** and **F1** scores are all achieved by the Random Forest model at 0.821. This means that this model manages to have a better ratio of true predicted positives to all predicted positives, a better ratio of true predicted positives to the actual number of positives in the dataset, and a better balance between precision and recall, across all classes.

The evaluation metrics here point to the Random Forest as a more approapriate model for this dataset in comparison to the decision tree, regardless of whether the target is less false positive predictions or less false negative predictions. However, the two models perform quite similarly.

## Final discussion

1. Structure: Decision trees make predictions based on a single tree, as it can also be observed in the plotted figure. On the other hand, the Random Forest model makes predictions based on multiple trees which are then combined for the final prediction. This might be the reason why the Random Forest was found to perform better across all metrics, except for **accuracy**.



2. The decision tree classifier is better when interpretability is important, as one can explain de model's decisions by interpreting the single decision tree. On the other hand, the Random Forest classifier has a better balance between precision and recall, which is why it is more suitable for complex data. In our dataset, some of the explanatory variables such as $X6-X11$ are slightly more complex. At the same time, correctly predicted true positives and true negatives can be considered equally important. Thus, we strive for the highest **F1** score to obtain the highest balance, which is why we would choose the Random Forest model.

3. **Accuracy** is an appropriate evaluation metric when classes are balanced in terms of distribution. In this dataset, the classes are not distributed in a particularly balanced manner. Hence, the **accuracy** score, although higher for the decision tree, is less important than the other evaluation metrics, where the Random Forest models performed better.
